In [1]:
from astropy.coordinates import SkyCoord
import astropy.coordinates as coord

import astropy.units as u
import matplotlib.pyplot as plt
import numpy as np
import os

from astropy import wcs
from astropy.io import fits
import sys

# from astroquery.simbad import Simbad
# from astroquery.vizier import Vizier

from astropy.table import Table, vstack
from astropy.table import Column

import collections
import subprocess
from astropy.io import ascii
import time
from astropy.utils.data import clear_download_cache
from astroquery.gaia import Gaia
# import warnings
# warnings.filterwarnings('ignore')
import pandas as pd

Created TAP+ (v1.0.1) - Connection:
	Host: gea.esac.esa.int
	Use HTTPS: False
	Port: 80
	SSL Port: 443


In [2]:
def findMatchesNoCondition(tab,match,max_sep = 5 * u.arcsec):
    
    df = tab.to_pandas()
    bigdf = match.to_pandas()
    
    number1 = match['index']
    ra1 = match['RA']
    dec1 = match['DEC']

    number2 = tab['Name'] # difference image
    ra2 = tab['RA(deg)']
    dec2 = tab['Dec(deg)']    
    
    cat1 = SkyCoord(ra=ra1*u.degree, dec=dec1*u.degree)
    cat2 = SkyCoord(ra=ra2*u.degree, dec=dec2*u.degree)
    
    idx, d2d, d3d = cat2.match_to_catalog_sky(cat1)
    sep_constraint = d2d < max_sep
    objectsFound = number2[sep_constraint]
    matchdf = df[df['Name'].isin(objectsFound)]
    matchTab = Table.from_pandas(matchdf)

    idx, d2d, d3d = cat1.match_to_catalog_sky(cat2)
    sep_constraint = d2d < max_sep
    #istrue = [x for x in sep_constraint if x == True]
    objectsFound = number1[sep_constraint]
    origdf = bigdf[bigdf['index'].isin(objectsFound)]
    origTab = Table.from_pandas(origdf)
    
    return [matchTab,origTab]

In [3]:
def findMatchesNoCondition(tab,match,max_sep = 2.5 * u.arcsec):
    
    df = tab.to_pandas()
    bigdf = match.to_pandas()
    
    number1 = tab['NUMBER'] # difference image
    ra1 = tab['RA(deg)']
    dec1 = tab['Dec(deg)']    
    
    number2 = match['NUMBER']
    ra2 = match['RA']
    dec2 = match['DEC']
    
    #makeDS9RegFile([tab],'/home/mj1e16/ds9RegFiles/foundinBoth','sex',colour='red')

    cat1 = SkyCoord(ra=ra1*u.degree, dec=dec1*u.degree)
    cat2 = SkyCoord(ra=ra2*u.degree, dec=dec2*u.degree)
    
    idx, d2d, d3d = cat1.match_to_catalog_sky(cat2)
    sep_constraint = d2d < max_sep
    
    objectsFound = number1[sep_constraint] # index of objects found in difference image
    objectsFoundList  = objectsFound.tolist()
    
    matches = idx[sep_constraint] # index of objects found in original,  image
    origdf = df[df['NUMBER'].isin(objectsFound)]


    finalTab = Table.from_pandas(origdf)

    return finalTab

In [4]:
cvTab = ascii.read('/home/mj1e16/Downloads/CVs_masterlist_20180424.csv')
#kepler = ascii.read('/home/mj1e16/outTables15/firstPassTransients15.csv')
kepler = ascii.read('/home/mj1e16/outTablesImproved/final/magone.csv')

In [5]:
cvTab[0:5]

Name,RA,Dec,RA(deg),Dec(deg),Constellation,Type,Period(d),MagBright(approx),MagFaint(approx)
str39,str12,str17,float64,float64,str11,str18,float64,float64,str18
CSS 100910:000025+332543,00 00 24.61,+33 25 43.1,0.10254167,33.42864,And,UGSU,--,15.9,20.5 CV
Gaia17ccg,00 00 42.30,+46 42 14.7,0.17625,46.704082,And,UG:,--,17.9,<22 G
BE Oct,00 00 49.02,-77 18 57.8,0.20425,-77.316055,Oct,UGSU,0.0747,15.0,19.4 V
MASTER OTJ000055.37+540128.5,00 00 55.37,+54 01 28.5,0.23070833,54.024582,--,CV?,--,16.5,--
Gaia17cue,00 01 21.23,+58 25 23.0,0.33845833,58.423054,Cas,UG:,--,18.1,<22 R


In [6]:
indexes = range(len(kepler))
kepler['NUMBER'] = indexes

indexes = range(len(cvTab))
cvTab['NUMBER'] = indexes

In [7]:
tabs = findMatchesNoCondition(cvTab,kepler)

In [8]:
tabs

Name,RA,Dec,RA(deg),Dec(deg),Constellation,Type,Period(d),MagBright(approx),MagFaint(approx),NUMBER
str29,str11,str11,float64,float64,str3,str4,float64,float64,str8,float64
V0344 Lyr,18 44 39.18,+43 22 28.0,281.16324,43.374443,Lyr,UGSU,0.087904,13.8,19 V,6287.0
MASTER OT J185838.73+460207.2,18 58 38.71,+46 02 07.2,284.6613,46.03533,Lyr,UGSU,--,15.8,20.3 B,6415.0
MASTER OT J191156.83+403101.6,19 11 56.83,+40 31 01.6,287.9868,40.517113,Lyr,UG,--,16.4,<18.4 CR,6533.0
ASASSN-15mt,19 12 35.54,+50 34 31.3,288.14807,50.57536,Cyg,UGSU,--,13.7,19.6 V,6536.0
V0585 Lyr,19 13 58.43,+40 44 08.8,288.49347,40.73578,Lyr,UGSU,--,14.9,17.0: B,6546.0
V1504 Cyg,19 28 56.45,+43 05 36.4,292.2352,43.093445,Cyg,UGSU,0.06951,13.5,17.4 p,6680.0


In [38]:
for x in tabs:
    ascii
    print(len(x))

In [44]:
ascii.write(tabs,'/home/mj1e16/outTablesImproved/cv10sn.csv',format='csv',fast_writer=False)
#ascii.write(tabs[1],'/home/mj1e16/outTablesImproved/cvkep25.csv',format='csv',fast_writer=False)